In [288]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)

# Remove Unnecessary Columns

In [289]:
# df = pd.read_pickle('CountVectorized.pickle')
# df['Job Title'] = df['New Job Title']
# df['Job Description'] = df['Modified_JobDesc']
# df.drop(['New Job Title', 'Easy Apply', 'Modified_JobDesc', 'languages_filtered', 'empathy', 'express',
#          'foundation', 'metal', 'metric', 'mockup', 'usability', 'tokens', 'tokens_filtered'], inplace=True, axis=1)
# df.to_pickle('CountVectorized.pickle')

# Read Dataset

In [290]:
df = pd.read_pickle("CountVectorized.pickle")

In [291]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,min_Salary,max_Salary,Job Position,analysis,analytic,azure,blockchain,cloud,collaborate,database,design,excel,framework,git,hadoop,java,javascript,linux,mathematic,mobile,network,prototype,python,react,rlanguage,security,sql,statistic,swift,tableau,test,visual,Chinese,English,French,German,Indonesian,Italian,Japanese,Korean,Portuguese,Russian,Spanish
0,Migration Data Analyst,51500.0,eager roll sleeve harness data drive policy ch...,3.2,Vera Institute of Justice,New York,New York,201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,5.000000e+08,37000,66000,Clerk,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,Product Quality Data Analyst,51500.0,overview provides analytical technical support...,3.8,Visiting Nurse Service of New York,New York,New York,10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,5.000000e+09,37000,66000,Clerk,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Service & Operations Data Analyst,51500.0,looking senior data analyst love mentorship da...,3.4,Squarespace,New York,New York,1001 to 5000 employees,Company - Private,Internet,Information Technology,5.000000e+08,37000,66000,Senior,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Data Analyst,51500.0,requisition numberrr remote yes collaborate. c...,4.1,Celerity,New York,McLean,201 to 500 employees,Subsidiary or Business Segment,IT Services,Information Technology,1.000000e+08,37000,66000,Clerk,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Data Analyst,51500.0,fanduel group fanduel group world class team b...,3.9,FanDuel,New York,New York,501 to 1000 employees,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",5.000000e+08,37000,66000,Reporter,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


# Import NLTK Library

In [292]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

## Clear unnecessary characters from Job Description

In [293]:
df['Job Description'] = df['Job Description'].apply(lambda x: re.sub(" +", " ", re.sub('[^a-zA-Z \n\.]'," ", " ".join(x.split("\n")))).lower())

### Remove Stop Words

In [294]:
stop = stopwords.words('english')
temp = df['Job Description']
only_text = temp.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

### Lemmatize the text

In [295]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
only_text = only_text.apply(lambda x : [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])
only_text = only_text.apply(lambda x : " ".join(x))

In [296]:
df['Modified_JobDesc'] = only_text
del only_text
df['Job Description'] = df['Modified_JobDesc']
df.drop('Modified_JobDesc', axis=1, inplace=True)

### Getting Linear Kernel and Cosine Similarity

In [297]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

### Creating a skills Column

In [298]:
dropping = ['analysis', 'analytic', 'azure', 'blockchain', 'cloud', 'collaborate',
       'database', 'design', 'excel', 'framework', 'git', 'hadoop', 'java',
       'javascript', 'linux', 'mathematic', 'mobile', 'network', 'prototype',
       'python', 'react', 'rlanguage', 'security', 'sql', 'statistic', 'swift',
       'tableau', 'test', 'visual', 'Chinese', 'English', 'French', 'German',
       'Indonesian', 'Italian', 'Japanese', 'Korean', 'Portuguese', 'Russian',
       'Spanish']
# all_skills = " ".join(dropping).lower()
df['Skills'] = ""
for col in df.columns[15:-1]:
    for i in range(df.shape[0]):
        if df.loc[i, col]:
            df.loc[i, 'Skills'] += " " + col.lower()   
df.drop(dropping, axis=1, inplace=True)
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,min_Salary,max_Salary,Job Position,Skills
0,Migration Data Analyst,51500.0,eager roll sleeve harness data drive policy ch...,3.2,Vera Institute of Justice,New York,New York,201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,5.000000e+08,37000,66000,Clerk,analysis database security statistic english
1,Product Quality Data Analyst,51500.0,overview provides analytical technical support...,3.8,Visiting Nurse Service of New York,New York,New York,10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,5.000000e+09,37000,66000,Clerk,analysis analytic database statistic english
2,Service & Operations Data Analyst,51500.0,looking senior data analyst love mentorship da...,3.4,Squarespace,New York,New York,1001 to 5000 employees,Company - Private,Internet,Information Technology,5.000000e+08,37000,66000,Senior,analysis analytic collaborate design excel en...
3,Data Analyst,51500.0,requisition numberrr remote yes collaborate. c...,4.1,Celerity,New York,McLean,201 to 500 employees,Subsidiary or Business Segment,IT Services,Information Technology,1.000000e+08,37000,66000,Clerk,analysis analytic design english
4,Data Analyst,51500.0,fanduel group fanduel group world class team b...,3.9,FanDuel,New York,New York,501 to 1000 employees,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",5.000000e+08,37000,66000,Reporter,analysis analytic database design network eng...


### Calculating Jaccard Coefficient

In [299]:
def Jaccard(x, y):
        """returns the jaccard similarity between two lists """
        x = x.split(" ")
        y = y.split(" ")
        intersection_cardinality = len(set(x).intersection(set(y)))
        union_cardinality = len(set(x).union(set(y)))
        return intersection_cardinality / float(union_cardinality)

### Calculating Similarity

In [315]:
def cal_similarity(cv, data=df):
        #similarity = []
        #for skills in gds.overall_dict:
        for i in data.index:
            skills=data.loc[i, 'Skills']
            #similarity.append(Jaccard(cv, skills))
            data.loc[i, 'similarity']=Jaccard(skills,cv)
        # gds['similarity'] = similarity
        col = ['Job Title', 'Salary Estimate', 'Rating', 'Company Name', 'Location', 'Size', 'Job Position', 'similarity']
        return data.sort_values(by='similarity', ascending=False).loc[:, col]

### Jaccard Coefficient for Recommendation

In [316]:
mycv=df.iloc[5]['Skills']
res=cal_similarity(mycv, data=df)
res.head()

,Job Title,Salary Estimate,Rating,Company Name,Location,Size,Job Position,similarity
44,Data Analyst,66500.0,3.0,Vettery,New York,201 to 500 employees,Clerk,1.0
5,Data Analyst,51500.0,3.9,Point72,New York,1001 to 5000 employees,Clerk,1.0
93,Data Analyst,69000.0,4.4,Tachyon Technologies,New York,201 to 500 employees,Clerk,1.0
811,Data Analyst,59000.0,4.8,Genesys Talent,Northbrook,1 to 50 employees,Clerk,1.0
2071,Data Analyst,60500.0,5.0,Concept Software & Services Inc,San Francisco,1 to 50 employees,Senior,0.8


### Cosine Similarity for Recommendation

In [317]:
def cosine_recommend(user_input, data=df):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_jobid = tfidf_vectorizer.fit_transform((data['Skills']))
    user_input = tfidf_vectorizer.transform([user_input])
    
    CosineSimilarity = {i:j for i, j  in enumerate(cosine_similarity(user_input, tfidf_jobid)[0])}
    CosineSimilarity = sorted(CosineSimilarity.items(), key=lambda x:x[1], reverse=True)[:10]
    CosineSimilarity = dict(CosineSimilarity)
    return list(CosineSimilarity.keys())

In [318]:
index = cosine_recommend(df.iloc[5]['Skills'], data=df)
new_df = df.iloc[index]
new_df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,min_Salary,max_Salary,Job Position,Skills,similarity
5,Data Analyst,51500.0,cubist cubist systematic strategy one world pr...,3.9,Point72,New York,Stamford,1001 to 5000 employees,Company - Private,Investment Banking & Asset Management,Finance,500000000.0,37000,66000,Clerk,collaborate database english,1.0
44,Data Analyst,66500.0,vettery changing way people hire get hired. us...,3.0,Vettery,New York,New York,201 to 500 employees,Company - Private,Internet,Information Technology,500000000.0,46000,87000,Clerk,collaborate database english,1.0
93,Data Analyst,69000.0,tachyon technology tachyon technology digital ...,4.4,Tachyon Technologies,New York,Irving,201 to 500 employees,Company - Private,IT Services,Information Technology,25000000.0,51000,87000,Clerk,collaborate database english,1.0
811,Data Analyst,59000.0,icon immediate need data analyst month contrac...,4.8,Genesys Talent,Northbrook,Houston,1 to 50 employees,Company - Private,Staffing & Outsourcing,Business Services,500000000.0,42000,76000,Clerk,collaborate database english,1.0
704,Health & Services Data Analyst,122500.0,providence calling senior clinical data analys...,3.4,Providence Health & Services,Gardena,Renton,10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,500000000.0,113000,132000,Senior,analysis collaborate database english,0.8
